# Outlier Handling

In [ ]:
num_columns = df.select_dtypes(exclude='object').columns

## Z-score based / Standard Scaler based method
# -----------------------
# def zscore_for_column(df,col):
#     #zscore = (val - mean) / std_dev
#     zscore = (df[col] - df[col].mean())/df[col].std()
#     return zscore


# # calculating zscore of each column and updating it in the copy.
# df_std = df_ohe.copy()
# for col in num_columns:
#     df_std[col] = zscore_for_column(df,col)

# df_std.describe()


# OR
#@@@@@@@@@@@@
# Z score / Standardscaler using library

df_std =df_ohe.copy()
from sklearn.preprocessing import StandardScaler
sd = StandardScaler()
sd.fit(df_std.loc[:,num_columns]) # it will learn --> mean and std_dev

# acually apply the formul and convert the data --> transform

df_std.loc[:,num_columns]=sd.transform(df_std.loc[:,num_columns])

df_std.describe()

# ---------------------------------

# Replacing the outliers with -3 and 3 (extreme values)

from IPython.utils.text import columnize


def outlier_imputation(df,col):
    df.loc[df[col] < -3 ,col] = -3
    df.loc[df[col] > 3 ,col] = 3
    return df
##########################################

for col in num_columns:
    df_std = outlier_imputation(df_std,col)


# --------------------------------------------

########################################################################
# # IQR Based outlier handling
# df_iqr = df_ohe.copy()
# q1, q3 = df['table'].quantile([0.25,0.75])
# def outlier_imputation_IQR(df,col):
#     q1, q3 = df[col].quantile([0.25,0.75])
#     iqr = q3 -q1

#     df.loc[df[col] < (q1-1.5*iqr),col ] = (q1-1.5*iqr)
#     df.loc[df[col] > (q3+1.5*iqr),col ] = (q3+1.5*iqr)
#     return df

# for col in num_columns:
#     df_iqr = outlier_imputation_IQR(df_iqr,col)

# df_iqr.describe()

########################################################################


# GridSearch

## Ridge Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0.01,0.5,1,5],
    'max_iter':[100,500,1000,5000]
}

# Ridge regression using grid search
gscv = GridSearchCV(Ridge(random_state=7),param_grid,
                scoring='r2',cv=2,verbose=2)

gscv.fit(X_train,Y_train)

print(gscv.best_score_)

print(gscv.best_params_)

# Create model with best parameters
rg = Ridge(alpha=0.01, max_iter=100, random_state=7)
rg.fit(x_train,y_train)
Y_pred= rg.predict(X_test)

from sklearn.metrics import r2_score

print(r2_score(Y_test,Y_pred))

# Predict on new data
X_test.iloc[0:1,:]
log_price = rg.predict(X_test.iloc[0:1,:])

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators':[50, 200] ,
    'max_samples': [0.6, 0.8]
}

gscv = GridSearchCV(RandomForestClassifier(max_depth = 4,
                                           oob_score=True,
                                           class_weight='balanced',
                                           random_state=7),
                    param_grid, cv=2, verbose=2)

gscv.fit(X_train_pca,y_train)

gscv.best_score_

gscv.best_params_

### Create RF  model using best params

rfc = RandomForestClassifier(n_estimators = 200,
                       max_samples= 0.6,
                        max_depth = 4,
                        oob_score=True,
                        class_weight='balanced',
                        random_state=7)

rfc.fit(X_train_pca,y_train)

y_pred_test = rfc.predict(X_test_pca)

from sklearn.metrics import classfication_report
classification_report(y_test, y_pred_test)

In [ ]:
#
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {
    "C" : [0.1, 2],
    "gamma" : [0.5, 1]
}

gscv = GridSearchCV(SVC(random_state=7), param_grid, cv=2, verbose=2)


gscv.fit(X_train_pca,y_train)
gscv.best_score_
gscv.best_params_

# Simple logistic regression

In [ ]:
import pandas as pd
df = pd.read_csv("BankChurners_cleaned.csv")

X = df.drop("Attrition_Flag",axis = 1)
Y = df["Attrition_Flag"]

X_ohe = pd.get_dummies(X)

Y.value_counts()

Y.loc[Y=='Existing Customer'] = 0
Y.loc[Y=='Attrited Customer'] = 1

Y.value_counts()
Y.dtype
Y = Y.astype("int")
Y.dtype

# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_ohe,Y,test_size = 0.3 ,random_state = 7,stratify = Y)

# Apply Logistic regression / logit classifier
###### class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)[source]¶

#n_jobs = if you want to run multiclass classifier, or how many threads you want to run
#l1_ratio

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'elasticnet',class_weight = 'balanced',random_state = 7, solver = 'saga',max_iter = 100,l1_ratio = 0.5)
# elasticnet = 0.5l1 +0.5l2

lr.fit(X_train,Y_train)

# convergenceWarning = loss is not minimum, till now the best boundary is not found
# model convergence = convergence = means achieving optimial soln. here achieving minimum loss.

Y_pred = lr.predict(X_test)

# Evaluation
## confusion matrix
### sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)

from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test,Y_pred)

# true negative false positive
# false negative true positive

# true positive is 0 becuse our model is not converged yet.





# End to End classification

In [ ]:
import pandas as pd

# Check The Data given by Competetion
#training data : all columns
#Consider training data as full data and train test split on it
#test data : target columns missing : can't be used to evaluate our model

train_df = pd.read_csv("train_jRxnrHD.csv")
train_df.info()
test_df = pd.read_csv("test_QaJU1Mh.csv")
test_df.info()

# Split the data

data = train_df.copy()

## Here target is in column 12 'target'
#Premium is another target column which can be predicted for regression problem / multi-class problem

X = data.drop(['premium', 'target'], axis=1)
y = data.loc[:,'target']

X.shape, y.shape

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,
                 test_size=0.3,
                 random_state=7,
                 stratify=y)

X_train.shape,X_test.shape,y_train.shape,y_test.shape

y.value_counts(normalize=True)
y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)

# Preprocessing

#- Remove the id columns or columns with unique values
#- here remove id column
#- Remove columns which contain ONLY one value
#- Remove columns / rows containing NAs more than threshold
#- Fill missing values with median
#- Encode the string columns


## Remove the id columns or columns with unique values

drop_columns = ['id']
X_train.drop(drop_columns,axis=1,inplace=True)

## columns which contain ONLY one value

#NO columns

X_train.nunique()

## columns / rows containing NAs more than threshold

#NO columns

X_train.isna().sum()/ X_train.shape[0]

## Fill missing values with median

X_train.median()
X_train.mode() #it will be used if na values are there in string type columns

fill_value = X_train.median()
X_train.fillna(fill_value, inplace=True)

# Encode the string columns

#- ONE hot encoding

X_train.select_dtypes(include='object').columns
X_train = pd.get_dummies(X_train)
X_train.shape
X_train.columns

## Standardization

##ONLY on numeric columns and not for categorical columns

cat_columns = ['sourcing_channel', 'residence_area_type']
num_columns = ['perc_premium_paid_by_cash_credit', 'age_in_days', 'Income',
       'Count_3-6_months_late', 'Count_6-12_months_late',
       'Count_more_than_12_months_late', 'application_underwriting_score',
       'no_of_premiums_paid']

from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train.loc[:, num_columns])

X_train_sc = X_train.copy()
std.fit(X_train_sc.loc[:, num_columns])
X_train_sc.loc[:, num_columns] = std.transform(X_train_sc.loc[:, num_columns])
X_train_sc.shape, X_train.shape


# Outlier Handling
#- impute the outliers in numeric columns using IQR method

def outlier_imputation(df,col):
    df.loc[df[col] < -3 ,col] = -3
    df.loc[df[col] > 3 ,col] = 3
    return df

for col in num_columns:
    df_std = outlier_imputation(df_std,col)



# Feature Selection

#- PCA
#- RFE
#- SelectFromModel ( Decision Tree)

## PCA

from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99, random_state=7)
pca.fit(X_train_sc)
X_train_pca = pca.transform(X_train_sc)
X_train_pca.shape

# Train the model

#- SVM
#fine tune C and gamma parameters
#- RandomForest
# n_estimators , max_samples , max_depth needs to be tuned
#- CatBoost

## SVM classifier ( SVC)

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {
    "C" : [0.1, 2],
    "gamma" : [0.5, 1]
}

gscv = GridSearchCV(SVC(random_state=7), param_grid, cv=2, verbose=2)
gscv.fit(X_train_pca,y_train)
gscv.best_score_
gscv.best_params_

## RandomForest

from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators':[50, 200] ,
    'max_samples': [0.6, 0.8]
}

gscv = GridSearchCV(RandomForestClassifier(max_depth = 4,
                                           oob_score=True,
                                           class_weight='balanced',
                                           random_state=7),
                    param_grid, cv=2, verbose=2)

gscv.fit(X_train_pca,y_train)

gscv.best_score_
gscv.best_params_

### Create RF  model using best params

rfc = RandomForestClassifier(n_estimators = 200,
                       max_samples= 0.6,
                        max_depth = 4,
                        oob_score=True,
                        class_weight='balanced',
                        random_state=7)

rfc.fit(X_train_pca,y_train)

### predict on train data only because test data is not preprocessed

y_pred_train = rfc.predict(X_train_pca)

from sklearn.metrics import f1_score
f1_score(y_train, y_pred_train)

# Make Test Data Ready fore predictions
#same steps of preprocessing as that of training data are done
#same features are selected here


X_test.drop(drop_columns, axis=1,inplace=True)
X_test.fillna(fill_value, inplace=True)
X_test_ohe = pd.get_dummies(X_test)
#X_test_ohe.shape, X_test.shape
X_test_ohe.loc[:, num_columns] = std.transform(X_test_ohe.loc[:, num_columns])
#X_test_ohe.shape
X_test_ohe.describe()

### feature selection using PCA for test data

X_test_pca = pca.transform(X_test_ohe)
#X_test_pca.shape

# prediction on Test data

y_pred = rfc.predict(X_test_pca)

f1_score(y_test,y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

# Apply the model to predict on new data
#- apply all the preprocessing steps
#- apply feature selection steps


rfc.predict(X_test_pca[:1])
rfc.predict_proba(X_test_pca[:1])

